# Petunjuk

Buatkan komparasi kondisi awal dan sesudah modifikasi sesuai dengan tugas yang di berikan, lalu buatkan code baru dengan modifikasi sesuaian dengan soal yang diberikan. kunci analisis akan menjadi tes interview setelah berhasil mendapatkan hasil komparasi. setiap poin akan di jumlahkan dan menjadi nilai tugas individu.

# Soal 1 [20 poin]

Tingkatkan performa GRU dengan mengubah fungsi aktivasi dan mengintegrasikan fitur embedding dari pre-trained model seperti Word2Vec.

Tugas:

1. Ubah fungsi aktivasi dari tanh menjadi ReLU.
2. Gunakan embedding dari Word2Vec untuk merepresentasikan data input.

Kunci Analisis: Lakukan perbandingan performa model sebelum dan sesudah modifikasi. Berikan comment anda terhadap akurasinya, di bandingkan yang ada di modul.

Bahan untuk melakukan eksperimen:

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from gensim.models import Word2Vec

# Data dummy (ganti dengan data sebenarnya dari modul)
sentences = [
    "ini adalah contoh kalimat pertama".split(),
    "chatbot membutuhkan evaluasi dan metrik".split(),
    "performa model harus dibandingkan".split(),
]

# 1. Word2Vec Embedding
# Membuat embedding Word2Vec
word2vec = Word2Vec(sentences, vector_size=50, min_count=1, workers=4)
vocab_size = len(word2vec.wv)

# Membuat word-index mapping
word_index = {word: i for i, word in enumerate(word2vec.wv.index_to_key)}

# Mengubah data ke bentuk indeks
def sentence_to_indices(sentence):
    return [word_index[word] for word in sentence]

data = np.array([sentence_to_indices(sentence) for sentence in sentences])
labels = np.array([0, 1, 1])  # Contoh label

# Padding sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = max(len(seq) for seq in data)
data_padded = pad_sequences(data, maxlen=max_length, padding="post")

# 2. Membuat Model GRU dengan tanh (Baseline)
model_tanh = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, weights=[word2vec.wv.vectors], trainable=False),
    GRU(128, activation='tanh', return_sequences=False),
    Dense(1, activation='sigmoid')
])

model_tanh.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training Baseline
print("Training Model dengan tanh")
model_tanh.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)

# Evaluasi Baseline
loss_tanh, acc_tanh = model_tanh.evaluate(data_padded, labels, verbose=0)
print(f"Accuracy dengan tanh: {acc_tanh:.4f}")

# 3. Modifikasi GRU dengan ReLU
model_relu = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, weights=[word2vec.wv.vectors], trainable=False),
    GRU(128, activation='relu', return_sequences=False),
    Dense(1, activation='sigmoid')
])

model_relu.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training Model dengan ReLU
print("Training Model dengan ReLU")
model_relu.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)

# Evaluasi dengan ReLU
loss_relu, acc_relu = model_relu.evaluate(data_padded, labels, verbose=0)
print(f"Accuracy dengan ReLU: {acc_relu:.4f}")

# 4. Analisis
print("\n--- Analisis ---")
print(f"Accuracy Baseline (tanh): {acc_tanh:.4f}")
print(f"Accuracy dengan ReLU: {acc_relu:.4f}")
if acc_relu > acc_tanh:
    print("Model dengan ReLU memiliki performa lebih baik.")
else:
    print("Model baseline (tanh) memiliki performa lebih baik.")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

# Soal 2 [40 poin]

Gabungkan Bidirectional LSTM dengan GRU dalam satu arsitektur hibrid.

Tugas:

1. Bangun model yang memiliki kombinasi layer Bidirectional LSTM dan GRU, **buatkan lebih dari satu lapisan**
2. Eksperimen dengan berbagai ukuran hidden layer (64, 128, dan 256).

Kunci Analisis: Evaluasi pengaruh ukuran hidden layer terhadap akurasi prediksi. lihat sesudah dilakukan eksperimen dengan yang ada di modul.

Bahan untuk Melakukan Eksperimen:

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

# Data dummy (ganti dengan data dari modul)
sentences = [
    "model deep learning memerlukan eksperimen".split(),
    "bidirectional LSTM memproses data sekuensial".split(),
    "kombinasi dengan GRU meningkatkan performa".split(),
]

labels = np.array([0, 1, 1])  # Label dummy

# 1. Word2Vec Embedding
word2vec = Word2Vec(sentences, vector_size=50, min_count=1, workers=4)
vocab_size = len(word2vec.wv)

# Membuat word-index mapping
word_index = {word: i for i, word in enumerate(word2vec.wv.index_to_key)}

# Mengubah data ke bentuk indeks
def sentence_to_indices(sentence):
    return [word_index[word] for word in sentence]

data = np.array([sentence_to_indices(sentence) for sentence in sentences])

# Padding sequences
max_length = max(len(seq) for seq in data)
data_padded = pad_sequences(data, maxlen=max_length, padding="post")

# 2. Model Hibrid: Bidirectional LSTM dan GRU
def build_hybrid_model(hidden_size):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=50, weights=[word2vec.wv.vectors], trainable=False),
        Bidirectional(LSTM(hidden_size, return_sequences=True)),
        GRU(hidden_size, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 3. Eksperimen dengan berbagai hidden layer size
hidden_sizes = [64, 128, 256]
results = {}

for size in hidden_sizes:
    print(f"\nTraining model dengan hidden size: {size}")
    model = build_hybrid_model(size)
    history = model.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)
    loss, accuracy = model.evaluate(data_padded, labels, verbose=0)
    results[size] = accuracy
    print(f"Accuracy dengan hidden size {size}: {accuracy:.4f}")

# 4. Analisis
print("\n--- Analisis ---")
for size, acc in results.items():
    print(f"Hidden size {size}: Accuracy = {acc:.4f}")

best_size = max(results, key=results.get)
print(f"Model dengan hidden size terbaik adalah {best_size} dengan akurasi {results[best_size]:.4f}")



Training model dengan hidden size: 64
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.6111 - loss: 0.6921
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6111 - loss: 0.6868 
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6111 - loss: 0.6811
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6111 - loss: 0.6753
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6111 - loss: 0.6688
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6111 - loss: 0.6623
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6111 - loss: 0.6550
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6111 - loss: 0.6471
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6111 - loss: 0.6398
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6111 - loss: 0.6342
Accuracy dengan hidden size 64: 0.6667

Training model dengan hidden size: 128
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step -

1. bangun model yang memiliki kombinasi layer bidirectional LSTM dan GRU, buatkan lebih dari satu lapisan
2. eksperimen dengan berbagai ukuran hidden layer (64, 128, dan 256)

dan analisislah dengan kunci = evaluasi pengarus ukuran hidden layer terhadap akurasi prediksi. lihatlah sesudah dilakukan eksperimen dengan yang ada di modul

jawablah tugas diatas dengan mengubah kode dibawah ini

# Soal 3 [40 poin]

Bagaimana arsitektur Bidirectional memengaruhi pemahaman konteks dalam sekuens data?

Tugas:
Eksperimen dengan model Bidirectional LSTM dengan dan tanpa pre-trained embedding (seperti GloVe).

Kunci Analisis: Apakah penggunaan pre-trained embedding memberikan keuntungan signifikan dibandingkan tanpa embedding? Jelaskan hasil eksperimen Anda (berikan comment atau markdown).

Bahan untuk melakukan eksperimen:

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Data dummy (ganti dengan data dari modul)
sentences = [
    "pemahaman konteks dalam data sekuensial sangat penting",
    "bidirectional LSTM membaca data dari dua arah",
    "penggunaan embedding pretrained dapat meningkatkan akurasi",
]

labels = np.array([0, 1, 1])  # Label dummy

# 1. Tokenisasi dan Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
data = tokenizer.texts_to_sequences(sentences)
max_length = max(len(seq) for seq in data)
data_padded = pad_sequences(data, maxlen=max_length, padding="post")

# 2. Memuat GloVe Pre-trained Embedding
embedding_index = {}
glove_file = "glove.6B.50d.txt"  # Pastikan file ini tersedia
with open(glove_file, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

# Membuat matriks embedding
embedding_dim = 50
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 3. Model dengan Pre-trained GloVe
def build_model_with_glove():
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  weights=[embedding_matrix], trainable=False),
        Bidirectional(LSTM(128, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 4. Model tanpa Pre-trained Embedding
def build_model_without_glove():
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, trainable=True),
        Bidirectional(LSTM(128, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Training dan evaluasi model
print("\n--- Model dengan Pre-trained GloVe ---")
model_with_glove = build_model_with_glove()
model_with_glove.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)
loss_with_glove, acc_with_glove = model_with_glove.evaluate(data_padded, labels, verbose=0)
print(f"Accuracy dengan pre-trained embedding GloVe: {acc_with_glove:.4f}")

print("\n--- Model tanpa Pre-trained Embedding ---")
model_without_glove = build_model_without_glove()
model_without_glove.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)
loss_without_glove, acc_without_glove = model_without_glove.evaluate(data_padded, labels, verbose=0)
print(f"Accuracy tanpa pre-trained embedding: {acc_without_glove:.4f}")

# Analisis
print("\n--- Analisis ---")
print(f"Accuracy dengan pre-trained GloVe: {acc_with_glove:.4f}")
print(f"Accuracy tanpa pre-trained embedding: {acc_without_glove:.4f}")
if acc_with_glove > acc_without_glove:
    print("Penggunaan pre-trained GloVe memberikan keuntungan dalam akurasi.")
else:
    print("Model tanpa pre-trained embedding memiliki performa serupa atau lebih baik.")


**Contoh Output**

--- Model dengan Pre-trained GloVe ---
Accuracy dengan pre-trained embedding GloVe: 0.9000

--- Model tanpa Pre-trained Embedding ---
Accuracy tanpa pre-trained embedding: 0.8500

--- Analisis ---
Model dengan pre-trained GloVe memberikan akurasi lebih tinggi dibandingkan model tanpa pre-trained embedding.
Hal ini menunjukkan bahwa embedding pre-trained memiliki informasi semantik yang mempercepat proses belajar dan meningkatkan performa model.


In [ ]:
#nomor 1
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences # Import pad_sequences here

# Data dummy (ganti dengan data sebenarnya dari modul)
sentences = [
    "ini adalah contoh kalimat pertama".split(),
    "chatbot membutuhkan evaluasi dan metrik".split(),
    "performa model harus dibandingkan".split(),
 ]

# 1. Word2Vec Embedding
# Membuat embedding Word2Vec
word2vec = Word2Vec (sentences, vector_size=50, min_count=1, workers=4)
vocab_size = len(word2vec.wv)

#Membuat word-index mapping
word_index = {word: i for i, word in enumerate (word2vec.wv.index_to_key)}

# Mengubah data ke bentuk indeks
def sentence_to_indices(sentence):
  return [word_index[word] for word in sentence]

# Convert sentences to indices
data = [sentence_to_indices(sentence) for sentence in sentences]

# Padding sequences to ensure uniform length
max_length = max(len(seq) for seq in data)  # Calculate max length
data_padded = pad_sequences(data, maxlen=max_length, padding="post") # Pad the sequences

# Now convert to NumPy array
data = np.array(data_padded) # Assign the padded data to 'data'

labels = np.array([0, 1, 1]) # Contoh Label



# 2. Membuat Model GRU dengan tanh (Baseline)
model_tanh = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, weights=[word2vec.wv.vectors]),
    GRU(128, activation= 'tanh', return_sequences=False),
    Dense(1, activation='sigmoid')
])
model_tanh.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training Baseline
print("Training Model dengan tanh")
model_tanh.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)

#Evaluasi Baseline
loss_tanh, acc_tanh = model_tanh.evaluate(data_padded, labels, verbose=0)
print (f"Accuracy dengan tanh: {acc_tanh:.4f}")

# 3. Modifikasi GRU dengan ReLU
model_relu = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, weights=[word2vec.wv.vectors]),
    GRU(128, activation='relu', return_sequences=False),
    Dense(1, activation='sigmoid')
])
model_relu.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training Model dengan ReLU
print("Training Model dengan ReLU")
model_relu.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)

#Evaluasi dengan ReLU
loss_relu, acc_relu = model_relu.evaluate(data_padded, labels, verbose=0)
print (f"Accuracy dengan ReLU: {acc_relu:.4f}")

#